# Modeling
evaluate model until predict

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 1000

%matplotlib inline

In [3]:
train = pd.read_pickle('trainx.pkl')
test = pd.read_pickle('test.pkl')

In [4]:
train.head()

,isSD,isSMP,isSMA,isSMK,isNegeri,provinsi,guru,siswa,rombongan,isK13,isKTSP,isPagi,isSiang,isSore,isMalam,isDouble,isPenuh,is5h,is6h,isInternet,isListrik,kelas,lab,perpustakaan,ngulang,baru,guru_pns,guru_honor,sertifikasi,sudah_sertifikasi,ptk,kelas_baik,kelas_rusak,sanitasi,akreditasi
0,1,0,0,0,1,18,14,321,12,1,0,1,0,0,0,0,0,0,1,1,1,12,0,0,5.0,53.0,11.0,3.0,14.0,10.0,2.0,8.0,4.0,3.0,2
1,1,0,0,0,1,9,25,596,20,1,0,0,0,0,0,1,0,0,1,0,1,13,0,1,25.0,109.0,22.0,0.0,25.0,20.0,3.0,10.0,3.0,10.0,1
2,1,0,0,0,1,11,9,149,6,1,0,1,0,0,0,0,0,0,1,0,1,6,0,0,5.0,34.0,5.0,3.0,9.0,2.0,2.0,0.0,6.0,0.0,2
3,1,0,0,0,0,13,7,264,11,1,0,1,0,0,0,0,0,0,1,0,1,12,1,1,2.0,31.0,0.0,1.0,7.0,0.0,0.0,12.0,0.0,2.0,1
4,0,0,1,0,0,0,33,674,20,0,1,1,0,0,0,0,0,0,1,1,1,20,0,0,0.0,229.0,0.0,0.0,33.0,6.0,0.0,1.0,19.0,0.0,2


## Using K-Fold to Evaluate

In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = {
    'LR': LogisticRegression(solver='lbfgs'),
    'KNN': KNeighborsClassifier(3),
    'DT': DecisionTreeClassifier(),
    'RF': RandomForestClassifier(n_estimators = 100, random_state=1),
    'ABC': AdaBoostClassifier(n_estimators = 100, random_state=1),
    'GB': GradientBoostingClassifier(n_estimators = 100, random_state=1),
    'GNB': GaussianNB(),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis()
}

def getScores(n_fold, train, model):
    kfold = KFold(n_fold, True, 1)
    
    targets = train['akreditasi']
    values = train.drop(columns=['akreditasi'])
    
    arr_score = []
    for trainX, testX in kfold.split(train):
        model.fit(values.iloc[trainX], targets.iloc[trainX])
        preds = model.predict(values.iloc[testX])

        arr_score.append(accuracy_score(targets.iloc[testX], preds))
        
    return arr_score

def scoringTable(n_fold, train, classifiers):
    table = pd.DataFrame()
    
    for model_name, model in classifiers.items():
        table[model_name] = getScores(n_fold, train, model)
    
    return table

In [6]:
table = scoringTable(10, train, classifiers)
table

,LR,KNN,DT,RF,ABC,GB,GNB,LDA,QDA
0,0.647598,0.576363,0.572405,0.672783,0.676021,0.680158,0.422378,0.662169,0.577982
1,0.670444,0.592013,0.573664,0.693470,0.695449,0.702105,0.437849,0.678000,0.618277
2,0.653355,0.587696,0.576363,0.682497,0.681957,0.686994,0.461774,0.663069,0.618457
3,0.659291,0.596330,0.574924,0.693290,0.682317,0.690952,0.457996,0.676381,0.631409
4,0.655693,0.588595,0.565030,0.683936,0.676201,0.694729,0.451340,0.665947,0.608922
5,0.662529,0.591473,0.593452,0.689153,0.682677,0.692750,0.446123,0.670804,0.619176
6,0.654012,0.576826,0.575207,0.679021,0.676682,0.685318,0.437028,0.662109,0.606693
7,0.654732,0.584563,0.569449,0.675063,0.674523,0.686218,0.440086,0.666967,0.583663
8,0.667866,0.599316,0.570349,0.689457,0.697553,0.702051,0.441706,0.679201,0.620007
9,0.653652,0.594638,0.559734,0.682080,0.676502,0.690356,0.480029,0.662648,0.624865


In [7]:
# See the describe() information to show more considerations
table.describe()

,LR,KNN,DT,RF,ABC,GB,GNB,LDA,QDA
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.657917,0.588781,0.573058,0.684075,0.681988,0.691163,0.447631,0.668729,0.610945
std,0.007109,0.007718,0.008807,0.007216,0.008213,0.007081,0.016024,0.006873,0.017403
min,0.647598,0.576363,0.559734,0.672783,0.674523,0.680158,0.422378,0.662109,0.577982
25%,0.653742,0.585346,0.569674,0.679786,0.676276,0.686412,0.438408,0.662754,0.607250
50%,0.655213,0.590034,0.573035,0.683216,0.679320,0.690654,0.443915,0.666457,0.618367
75%,0.661720,0.593982,0.575136,0.689381,0.682587,0.694235,0.456332,0.674987,0.619799
max,0.670444,0.599316,0.593452,0.693470,0.697553,0.702105,0.480029,0.679201,0.631409


### We see that RF and GB performs the best
We choose GradientBoostingClassifier to predict

In [8]:
model = GradientBoostingClassifier(n_estimators = 100, random_state=1)

model.fit(train.drop(columns=['akreditasi']), train['akreditasi'])
preds = model.predict(test)

In [9]:
sample = pd.read_csv('sample-submission-1.csv')
sample.head()

,id,akreditasi
0,55586,2
1,55587,2
2,55588,2
3,55589,2
4,55590,2


In [10]:
sample['akreditasi'] = preds

In [11]:
sample.head()

,id,akreditasi
0,55586,1
1,55587,1
2,55588,1
3,55589,2
4,55590,1


### We've done the prediction.
In the next level, we also need to do parameter tuning for better precision